# Network Measures

Generating metrics of walking edge lengths and intersection density for a hex grid

First we need to download OSM network data using OSMNX. The data is to big to download and process at once, so we do this in chunks, for each of the 17 census divisions in the region, and then patch it back together with some `ogr2ogr` merging.

In [1]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import osmnx as ox
import matplotlib.pyplot as plt

In [3]:
# read the shp with the boundaries
dfcd = gpd.read_file("../input_data/spatial_boundaries/CensusDivisions/2016/CensusDivisions_GGH_utm17n.shp")
# convert to WGS84 for download
dfcd = dfcd.to_crs(epsg=4326)

In [47]:
# setup the query for downloading data
filter_drive = ('["area"!~"yes"]'
            '["highway"!~"motor|trunk|foot|proposed|construction|abandoned|platform|raceway|foot|cycleway|path|secondary_link|primary_link"]'
            '["service"!~"driveway|emergency_access|parking_aisle|drive-through"]'
            '["access"!~"private"]')
filter_walk = ('["area"!~"yes"]'
            '["highway"!~"motor|trunk|proposed|construction|abandoned|platform|raceway"]'
            '["foot"!~"no"]'
            '["footway"!~"sidewalk"]'
            '["service"!~"private|parking_aisle"]'
            '["access"!~"private"]')

In [2]:
# loop over each census division boundary, downloading nodes and edges from the OSM network

# using just the streets for nodes, but all walking paths for edges

x = 0
while x < (len(dfcd)):
    print(x,name)
    boundary = dfcd.loc[x,"geometry"]
    name = dfcd.loc[x,"CDNAME"]
    x += 1
    
    G = ox.graph_from_polygon(boundary, custom_filter=filter_drive)
    G = ox.project_graph(G)
    
    # grab nodes and put into data-frame
    nodes_all = ox.graph_to_gdfs(G, edges=False)
    nodes_all = nodes_all.to_crs(epsg=32617)
    streets_per_node = ox.count_streets_per_node(G) # count streets per node - and join to the spatial file
    streets_per_node = pd.DataFrame.from_dict(data=streets_per_node,orient='index')
    streets_per_node.columns = ["streets_n"]
    nodes_all = nodes_all.join(streets_per_node)
    nodes_all = nodes_all.query('streets_n >= 3')
    nodes_all[['lat','lon','geometry','streets_n']]

    nodes_all.to_file(driver = 'ESRI Shapefile', filename= "temp/nodes/" + name + ".shp")
    
    G = ox.graph_from_polygon(boundary, custom_filter=filter_walk)
    G = ox.project_graph(G)

    edges_all = ox.graph_to_gdfs(G, nodes=False)
    edges_all = edges_all[["u","v","geometry"]]

    edges_all.to_file(driver = 'ESRI Shapefile', filename= "temp/edges/" + name + ".shp")
    


NameError: name 'dfcd' is not defined

Merge the nodes and edges in each folder using this `ogrg2ogr` snippet:

```sh
for file in *.shp
do
  if [ -f  merged.shp ]
    then
      ogr2ogr -f "ESRI Shapefile" merged.shp $file -update -append -dialect "SQLite" -sql "SELECT '${file%.*}' AS filename, * FROM ${file%.*}"
    else
      ogr2ogr -f "ESRI Shapefile" merged.shp $file -dialect "SQLite" -sql "SELECT '${file%.*}' AS filename, * FROM ${file%.*}"
  fi
done
```

Reproject in epsg 32617 (the UTM zone for the region)

```sh
ogr2ogr -t_srs EPSG:32617 merged_edges_utm.shp merged_edges.shp
ogr2ogr -t_srs EPSG:32617 merged_nodes_utm.shp merged_nodes.shp
```

Then send the merged data into the PostGIS database

```sh
shp2pgsql -I -s 32617 -W "latin1" input_data/osm_data/merged_edges_utm.shp OSM_edges_walk | psql -U ja -d urban_form_toronto

shp2pgsql -I -s 32617 -W "latin1" input_data/osm_data/merged_nodes_utm.shp OSM_nodes_streets | psql -U ja -d urban_form_toronto
```

## Edge Density

Generating the length of walking edges per hex grid in PostGIS

```sql
-- creating a spatial index for the sake of brevity

DROP INDEX IF EXISTS hex_grid_200m_gix;
CREATE INDEX hex_grid_200m_gix ON hex_grid_200m USING GIST (geom);

DROP INDEX IF EXISTS OSM_edges_walk_gix;
CREATE INDEX OSM_edges_walk_gix ON OSM_edges_walk USING GIST (geom);

-- intersect the edges with the grid, summing the edge length in each grid cells

DROP TABLE IF EXISTS out_data_hex_osmwalkedge2018;
CREATE TABLE out_data_hex_osmwalkedge2018 AS (
    SELECT
    hex_grid_200m.id AS hexid,
    coalesce(sum
        (ST_Length(
            ST_Intersection(OSM_edges_walk.geom,hex_grid_200m.geom)
        )),0) AS edge_length
    FROM OSM_edges_walk RIGHT JOIN hex_grid_200m ON 
    ST_Intersects(OSM_edges_walk.geom,hex_grid_200m.geom)
    GROUP BY hexid
);
             
-- output
    
\COPY out_data_hex_osmwalkedge2018 TO 'out_data_hex_osmwalkedge2018.csv' WITH (FORMAT CSV, HEADER);

```

## Intersection Density

Clustering Points

```sql

-- first we settin' up dat spatial index
DROP INDEX IF EXISTS OSM_nodes_streets_gix;
CREATE INDEX OSM_nodes_streets_gix ON OSM_nodes_streets USING GIST (geom);

-- cluster the nodes, as to not have several points at parrelel street intersections
-- built from 
-- https://gis.stackexchange.com/questions/11567/spatial-clustering-with-postgis
DROP TABLE IF EXISTS temp_cluster_nodes;
CREATE TABLE temp_cluster_nodes AS
(
SELECT 
  row_number() over () AS gid,
  -- gc AS geom_collection,
  ST_Buffer(ST_Centroid(gc),25) AS geom
  -- ST_MinimumBoundingCircle(gc) AS circle
  FROM (
  SELECT 
      unnest(ST_ClusterWithin(geom, 25)) AS gc
  FROM OSM_nodes_streets
) f
);

-- join and count the number of streets per intersection to the clustered points
DROP TABLE IF EXISTS OSM_nodes_streets_clustered;
CREATE TABLE OSM_nodes_streets_clustered AS (
SELECT
    sum(coalesce(OSM_nodes_streets.streets_n,0)) AS n_streets,
    ST_Centroid(temp_cluster_nodes.geom) AS geom,
    temp_cluster_nodes.gid AS gid                      
FROM
OSM_nodes_streets RIGHT OUTER JOIN temp_cluster_nodes ON ST_Intersects(OSM_nodes_streets.geom,temp_cluster_nodes.geom)
GROUP BY temp_cluster_nodes.gid, temp_cluster_nodes.geom
);

-- create spatial index on this new set of points
DROP INDEX IF EXISTS OSM_nodes_streets_clustered_gix;
CREATE INDEX OSM_nodes_streets_clustered_gix ON OSM_nodes_streets_clustered USING GIST (geom);

-- subset for equal to 3 streets, and greater than 3 streets
DROP TABLE IF EXISTS temp_i3;
CREATE TABLE temp_i3 AS (
    SELECT * FROM OSM_nodes_streets_clustered
    WHERE n_streets < 4
);
        
DROP TABLE IF EXISTS temp_i4;
CREATE TABLE temp_i4 AS (
    SELECT * FROM OSM_nodes_streets_clustered
    WHERE n_streets > 3
);
        
        
-- spatial index on these new points
DROP INDEX IF EXISTS temp_i3_gix;
CREATE INDEX temp_i3_gix ON temp_i3 USING GIST (geom);
DROP INDEX IF EXISTS temp_i4_gix;
CREATE INDEX temp_i4_gix ON temp_i4 USING GIST (geom);
   
        
-- count points per polygon (hex)
DROP TABLE IF EXISTS out_data_hex_int3way2018;
CREATE TABLE out_data_hex_int3way2018 AS (
SELECT
    coalesce(count(temp_i3.n_streets),0) AS int3way,
    hex_grid_200m.id AS hexid
FROM
temp_i3 RIGHT OUTER JOIN hex_grid_200m ON ST_Intersects(temp_i3.geom,hex_grid_200m.geom)
GROUP BY hexid
);
    
DROP TABLE IF EXISTS out_data_hex_int4way2018;
CREATE TABLE out_data_hex_int4way2018 AS (
SELECT
    coalesce(count(temp_i4.n_streets),0) AS int4way,
    hex_grid_200m.id AS hexid
FROM
temp_i4 RIGHT OUTER JOIN hex_grid_200m ON ST_Intersects(temp_i4.geom,hex_grid_200m.geom)
GROUP BY hexid
);
     
-- output the data 

\COPY out_data_hex_int4way2018 TO 'out_data_hex_int4way2018.csv' WITH (FORMAT CSV, HEADER);        
\COPY out_data_hex_int3way2018 TO 'out_data_hex_int3way2018.csv' WITH (FORMAT CSV, HEADER);
        
```